<a href="https://colab.research.google.com/github/numinousmuses/ei-keyword-spotting-project/blob/main/ei_audio_dataset_curation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Keyword Spotting Dataset Curation

[![Open In Colab <](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ShawnHymel/ei-keyword-spotting/blob/master/ei-audio-dataset-curation.ipynb)

Use this tool to download the Google Speech Commands Dataset, combine it with your own keywords, mix in some background noise, and upload the curated dataset to Edge Impulse. From there, you can train a neural network to classify spoken words and upload it to a microcontroller to perform real-time keyword spotting.

 1. Upload samples of your own keyword (optional)
 2. Adjust parameters in the Settings cell (you will need an [Edge Impulse](https://www.edgeimpulse.com/) account)
 3. Run the rest of the cells! ('shift' + 'enter' on each cell)



### Upload your own keyword samples
You are welcome to use my [custom keyword dataset](https://github.com/ShawnHymel/custom-speech-commands-dataset), but note that it's limited and that I can't promise it will work well. If you want to use it, uncomment the `###Download custom dataset` cell below. You may also add your own recorded keywords to the extracted folder (`/content/custom_keywords`) to augment what's already there.

If you'd rather upload your own custom keyword dataset, follow these instructions:

On the left pane, in the file browser, create a directory structure containing space for your keyword audio samples. All samples for each keyword should be in a directory with that keyword's name. 

The audio samples should be `.wav` format, mono, and 1 second long. Bitrate and bitdepth should not matter. Samples shorter than 1 second will be padded with 0s, and samples longer than 1 second will be truncated to 1 second. The exact name of each `.wav` file does not matter, as they will be read, mixed with background noise, and saved to a separate file with an auto-generated name. Directory name does matter (it is used to determine the name of the class during neural network training).

Right-click on each keyword directory and upload all of your samples. Your directory structor should look like the following:

```
/
|- content
|--- custom_keywords
|----- keyword_1
|------- 000.wav
|------- 001.wav
|------- ...
|----- keyword_2
|------- 000.wav
|------- 001.wav
|------- ...
|----- ...
```




In [ ]:
### Update Node.js to the latest stable version
!npm cache clean -f
!npm install -g n
!n stable

npm WARN using --force I sure hope you know what you are doing.
/tools/node/bin/n -> /tools/node/lib/node_modules/n/bin/n
+ n@9.0.0
added 1 package from 2 contributors in 0.423s


   ╭────────────────────────────────────────────────────────────────╮
   │                                                                │
   │      New major version of npm available! 6.14.8 → 8.15.0       │
   │   Changelog: https://github.com/npm/cli/releases/tag/v8.15.0   │
   │               Run npm install -g npm to update!                │
   │                                                                │
   ╰────────────────────────────────────────────────────────────────╯

  installing : node-v16.16.0
       mkdir : /usr/local/n/versions/node/16.16.0
       fetch : https://nodejs.org/dist/v16.16.0/node-v16.16.0-linux-x64.tar.xz
######################################################################## 100.0%
     copying : node/16.16.0
   installed : v16.16.0 (with npm 8.11.0)

Note: the node comma

In [ ]:
### Install required packages and tools
!python -m pip install soundfile
!npm install -g --unsafe-perm edge-impulse-cli

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.
npm WARN deprecated debug@4.1.1: Debug versions >=3.2.0 <3.2.7 || >=4 <4.3.1 have a low-severity ReDos regression when used in a Node.js environment. It is recommended you upgrade to 3.2.7 or 4.3.1. (https://github.com/visionmedia/debug/issues/797)
npm WARN deprecated debug@4.1.1: Debug versions >=3.2.0 <3.2.7 || >=4 <4.3.1 have a low-severity ReDos regression when used in a Node.js environment. It is recommended you upgrade to 3.2.7 or 4.3.1. (https://github.com/visionmedia/debug/issues/797)
npm WARN deprecated har-validator@5.1.5: this library is no longer supported
npm WARN deprecated uuid@3.4.0: Please upgrade  to version 7 or higher.  Older versions may use Math.random() in certain circumstances, which is known to be problematic.  See https://v8.dev/blog/math-random for details.
npm WARN dep

In [ ]:
### Settings (You probably do not need to change these)
BASE_DIR = "/content"
OUT_DIR = "keywords_curated"
GOOGLE_DATASET_FILENAME = "speech_commands_v0.02.tar.gz"
GOOGLE_DATASET_URL = "http://download.tensorflow.org/data/" + GOOGLE_DATASET_FILENAME
GOOGLE_DATASET_DIR = "google_speech_commands"
CUSTOM_KEYWORDS_FILENAME = "main.zip"
CUSTOM_KEYWORDS_URL = "https://github.com/ShawnHymel/custom-speech-commands-dataset/archive/" + CUSTOM_KEYWORDS_FILENAME
CUSTOM_KEYWORDS_DIR = "custom_keywords"
CUSTOM_KEYWORDS_REPO_NAME = "custom-speech-commands-dataset-main"
CURATION_SCRIPT = "dataset-curation.py"
CURATION_SCRIPT_URL = "https://raw.githubusercontent.com/ShawnHymel/ei-keyword-spotting/master/" + CURATION_SCRIPT
UTILS_SCRIPT_URL = "https://raw.githubusercontent.com/ShawnHymel/ei-keyword-spotting/master/utils.py"
NUM_SAMPLES = 1500    # Target number of samples to mix and send to Edge Impulse
WORD_VOL = 1.0        # Relative volume of word in output sample
BG_VOL = 0.1          # Relative volume of noise in output sample
SAMPLE_TIME = 1.0     # Time (seconds) of output sample
SAMPLE_RATE = 16000   # Sample rate (Hz) of output sample
BIT_DEPTH = "PCM_16"  # Options: [PCM_16, PCM_24, PCM_32, PCM_U8, FLOAT, DOUBLE]
BG_DIR = "_background_noise_"
TEST_RATIO = 0.2      # 20% reserved for test set, rest is for training
EI_INGEST_TEST_URL = "https://ingestion.edgeimpulse.com/api/test/data"
EI_INGEST_TRAIN_URL = "https://ingestion.edgeimpulse.com/api/training/data"

In [ ]:
### Download Google Speech Commands Dataset
!cd {BASE_DIR}
!wget {GOOGLE_DATASET_URL}
!mkdir {GOOGLE_DATASET_DIR}
!echo "Extracting..."
!tar xfz {GOOGLE_DATASET_FILENAME} -C {GOOGLE_DATASET_DIR}

--2022-07-26 13:46:39--  http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 108.177.13.128, 2607:f8b0:400c:c09::80
Connecting to download.tensorflow.org (download.tensorflow.org)|108.177.13.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2428923189 (2.3G) [application/gzip]
Saving to: ‘speech_commands_v0.02.tar.gz’

speech_commands_v0. 100%[===================>]   2.26G   172MB/s    in 13s     

2022-07-26 13:46:52 (176 MB/s) - ‘speech_commands_v0.02.tar.gz’ saved [2428923189/2428923189]

Extracting...


In [ ]:
### Pull out background noise directory
!cd {BASE_DIR}
!mv "{GOOGLE_DATASET_DIR}/{BG_DIR}" "{BG_DIR}"

In [ ]:
## (Optional) Download custom dataset--uncomment the code in this cell if you want to use my custom datase

# Download, extract, and move dataset to separate directory
!cd {BASE_DIR}
!wget {CUSTOM_KEYWORDS_URL}
!echo "Extracting..."
!unzip -q {CUSTOM_KEYWORDS_FILENAME}
!mv "{CUSTOM_KEYWORDS_REPO_NAME}/{CUSTOM_KEYWORDS_DIR}" "{CUSTOM_KEYWORDS_DIR}"

--2022-07-26 13:56:45--  https://github.com/ShawnHymel/custom-speech-commands-dataset/archive/main.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/ShawnHymel/custom-speech-commands-dataset/zip/refs/heads/main [following]
--2022-07-26 13:56:46--  https://codeload.github.com/ShawnHymel/custom-speech-commands-dataset/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.112.9
Connecting to codeload.github.com (codeload.github.com)|140.82.112.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘main.zip’

main.zip                [             <=>    ]  58.89M  8.82MB/s    in 11s     

2022-07-26 13:56:57 (5.14 MB/s) - ‘main.zip’ saved [61755655]

Extracting...


In [ ]:
### User Settings (do change these)

# Location of your custom keyword samples (e.g. "/content/custom_keywords")
# Leave blank ("") for no custom keywords. set to the CUSTOM_KEYWORDS_DIR
# variable to use samples from my custom-speech-commands-dataset repo.
CUSTOM_DATASET_PATH = "/content/custom_keywords"

# Edge Impulse > your_project > Dashboard > Keys
EI_API_KEY = "ei_081b0bc5d316fdf641587eef548eb7ed0409889d2609703a4739a20e8707b3f6" 

# Comma separated words. Must match directory names (that contain samples).
# Recommended: use 2 keywords for microcontroller demo
TARGETS = "goodnight, up"

In [ ]:
### Download curation and utils scripts
!wget {CURATION_SCRIPT_URL}
!wget {UTILS_SCRIPT_URL}

--2022-07-26 14:02:30--  https://raw.githubusercontent.com/ShawnHymel/ei-keyword-spotting/master/dataset-curation.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17427 (17K) [text/plain]
Saving to: ‘dataset-curation.py’

dataset-curation.py 100%[===================>]  17.02K  --.-KB/s    in 0s      

2022-07-26 14:02:30 (44.2 MB/s) - ‘dataset-curation.py’ saved [17427/17427]

--2022-07-26 14:02:31--  https://raw.githubusercontent.com/ShawnHymel/ei-keyword-spotting/master/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 

In [ ]:
### Perform curation and mixing of samples with background noise
!cd {BASE_DIR}
!python {CURATION_SCRIPT} \
  -t "{TARGETS}" \
  -n {NUM_SAMPLES} \
  -w {WORD_VOL} \
  -g {BG_VOL} \
  -s {SAMPLE_TIME} \
  -r {SAMPLE_RATE} \
  -e {BIT_DEPTH} \
  -b "{BG_DIR}" \
  -o "{OUT_DIR}" \
  "{GOOGLE_DATASET_DIR}" \
  "{CUSTOM_DATASET_PATH}"

/usr/local/lib/python3.7/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  _resample_loop_p(x, t_out, interp_win, interp_delta, num_table, scale, y)
-----------------------------------------------------------------------
Keyword Dataset Curation Tool
v0.1
-----------------------------------------------------------------------
Gathering random background noise snippets (1500 files)
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Mixing: goodnight (1500 files)
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Mixing: up (1500 files)
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Mixing: _unknown (1500 files)
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Done!


In [ ]:
### Use CLI tool to send curated dataset to Edge Impulse

!cd {BASE_DIR}

# Imports
import os
import random

# Seed with system time
random.seed()

# Go through each category in our curated dataset
for dir in os.listdir(OUT_DIR):
  
  # Create list of files for one category
  paths = []
  for filename in os.listdir(os.path.join(OUT_DIR, dir)):
    paths.append(os.path.join(OUT_DIR, dir, filename))

  # Shuffle and divide into test and training sets
  random.shuffle(paths)
  num_test_samples = int(TEST_RATIO * len(paths))
  test_paths = paths[:num_test_samples]
  train_paths = paths[num_test_samples:]

  # Create arugments list (as a string) for CLI call
  test_paths = ['"' + s + '"' for s in test_paths]
  test_paths = ' '.join(test_paths)
  train_paths = ['"' + s + '"' for s in train_paths]
  train_paths = ' '.join(train_paths)
  
  # Send test files to Edge Impulse
  !edge-impulse-uploader \
    --category testing \
    --label {dir} \
    --api-key {EI_API_KEY} \
    --silent \
    {test_paths}

  # # Send training files to Edge Impulse
  !edge-impulse-uploader \
    --category training \
    --label {dir} \
    --api-key {EI_API_KEY} \
    --silent \
    {train_paths}

Streaming output truncated to the last 5000 lines.
[ 701/1200] Uploading keywords_curated/up/up.1234.wav OK (273 ms)
[ 702/1200] Uploading keywords_curated/up/up.0812.wav OK (375 ms)
[ 703/1200] Uploading keywords_curated/up/up.1202.wav OK (374 ms)
[ 704/1200] Uploading keywords_curated/up/up.0509.wav OK (329 ms)
[ 705/1200] Uploading keywords_curated/up/up.0000.wav OK (391 ms)
[ 706/1200] Uploading keywords_curated/up/up.0313.wav OK (498 ms)
[ 707/1200] Uploading keywords_curated/up/up.1472.wav OK (304 ms)
[ 708/1200] Uploading keywords_curated/up/up.1323.wav OK (324 ms)
[ 709/1200] Uploading keywords_curated/up/up.0836.wav OK (333 ms)
[ 710/1200] Uploading keywords_curated/up/up.1149.wav OK (329 ms)
[ 711/1200] Uploading keywords_curated/up/up.0896.wav OK (420 ms)
[ 712/1200] Uploading keywords_curated/up/up.0083.wav OK (365 ms)
[ 713/1200] Uploading keywords_curated/up/up.0639.wav OK (286 ms)
[ 714/1200] Uploading keywords_curated/up/up.1271.wav OK (409 ms)
[ 715/1200] Uploading key